In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

In [4]:
from model.mrcnn.config import Config
from model.mrcnn import utils
import model.mrcnn.model as modellib
from model.mrcnn import visualize
from model.mrcnn.model import log
from model.mrcnn import data_generator as datagen
from model.mrcnn import image_meta as meta
from model.mrcnn import dataset

%matplotlib inline 

Using TensorFlow backend.


In [5]:
from data.openimage.oid_first import OIDataset

In [7]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(os.environ['HOME'], 'partial_experiments')

# Local path to trained weights file MEH
COCO_MODEL_PATH = os.path.join(os.environ['HOME'], "partial-labels/experiments/frcnn", "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [8]:
from keras.preprocessing import image as k_image
from keras.applications.imagenet_utils import preprocess_input

In [9]:
from collections import defaultdict

from pprint import pprint

## Configurations

In [ ]:
class OIDConfig(Config):
    """
    Configuration for training on the OID dataset.
    Derives from the base Config class and overrides values specific to OID
    """
    # Give the configuration a recognizable name
    NAME = "openimages"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    NB_CLASSES = 500 + 1

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 448
    IMAGE_MAX_DIM = 448

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (64, 128, 256)  # anchor side in pixels
    
    # STEPS_PER_EPOCH = 2000   # Size of the dataset
    STEPS_PER_EPOCH = 3000
    NB_EPOCH = 200
    
    # old config
    IMG_SIZE = 448
    DATASET_PATH = '/local/DEEPLEARNING/oid/'
    NB_CHANNELS = 3
    
config = OIDConfig()
config.display()

## Notebook Preferences

In [10]:
def get_ax(rows=1, cols=1, size=8):
    """
    Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## OID dataset

In [ ]:
# Training dataset
dataset_train = OIDataset()
dataset_train.load_oid(config.DATASET_PATH, 8, 'train', config)
dataset_train.prepare()

In [ ]:
# Validation dataset
dataset_val = OIDataset()
dataset_val.load_oid(config.DATASET_PATH, 8, 'val', config)
dataset_val.prepare()

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset_val.image_ids, 4)
print(image_ids)

In [ ]:
len(dataset_train.image_ids)

In [ ]:
dataset_val.sample_ids[55102]

In [ ]:
dataset_val.class_names

In [ ]:
#image_ids = [55102]
for image_id in image_ids:
    print('image id %s' % image_id)
    img = dataset_val.load_image(image_id)
    bboxes, class_ids = dataset_val.load_bboxes(image_id)
    bboxes = np.array(bboxes)
    print(bboxes.shape)
    print(bboxes[0])
    
    class_ids = np.array(class_ids)
    print(class_ids.shape)
    
    visualize.display_instances(img, bboxes, class_ids, dataset_val.class_names, figsize=(8, 8))

In [ ]:
pprint(dataset_val.class_names)

In [ ]:
pprint(dataset_val.class_data)

## Create Model

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", 
                          config=config,
                          model_dir=MODEL_DIR)

In [ ]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=config.NB_EPOCHS, 
            layers="all")

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
# model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes.h5")
# model.keras_model.save_weights(model_path)

## Detection

In [ ]:
class InferenceConfig(OIDConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
model_path = os.path.join('/home/caleml/partial_experiments/openimages20200114T1558/', 'mask_rcnn_openimages_0002.h5')
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
# model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:

image_id = random.choice(dataset_val.image_ids)
print(image_id)
original_image, image_meta, gt_class_id, gt_bbox = datagen.load_image_gt(dataset_val, inference_config, image_id)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)

visualize.display_instances(original_image, gt_bbox, gt_class_id, dataset_val.class_names, figsize=(8, 8))

In [ ]:
results = model.detect([original_image], verbose=1)

print(results)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['class_ids'], dataset_val.class_names, r['scores'], ax=get_ax())

## Evaluation

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
# image_ids = np.random.choice(dataset_val.image_ids, 100)
APs = []
count = 0

for image_id in dataset_val.image_ids:
    count += 1
    if count % 1000 == 0:
        print("done %s val images, temp AP %s" % (count, np.mean(APs)))
    
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox = datagen.load_image_gt(dataset_val, inference_config, image_id)
    
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    
    # Compute AP
    AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, r["rois"], r["class_ids"], r["scores"])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))

## Check OID generator is correctly rolling images over epochs

In [14]:
class OIDConfig(Config):
    """
    Configuration for training on the OID dataset.
    Derives from the base Config class and overrides values specific to OID
    """
    # Give the configuration a recognizable name
    NAME = "openimages"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 4

    # Number of classes (including background)
    NB_CLASSES = 500 + 1

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 448
    IMAGE_MAX_DIM = 448

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (64, 128, 256)  # anchor side in pixels
    
    # STEPS_PER_EPOCH = 2000   # Size of the dataset
    STEPS_PER_EPOCH = 10
    NB_EPOCH = 3
    
    # old config
    IMG_SIZE = 448
    DATASET_PATH = '/local/DEEPLEARNING/oid/'
    NB_CHANNELS = 3
    
config = OIDConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     4
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DATASET_PATH                   /local/DEEPLEARNING/oid/
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 4
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  448
IMAGE_META_SIZE                513
IMAGE_MIN_DIM                  448
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [448 448   3]
IMG_SIZE                       448
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_l

In [13]:
# Validation dataset
dataset_val = OIDataset()
dataset_val.load_oid(config.DATASET_PATH, 4, 'val', config)
dataset_val.prepare()

loaded annotations in 102.78576040267944
Registered images in 0.15045595169067383


In [15]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", 
                          config=config,
                          model_dir=MODEL_DIR)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


In [16]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

In [17]:
# head branch training with val dataset to see which ids are printed for each epoch
model.train(dataset_val, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=2, 
            layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: /home/caleml/partial_experiments/openimages20200116T1704/mask_rcnn_openimages_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_class_logits     (TimeDistributed)



/home/caleml/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/caleml/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



Epoch 1/2
yielding image id 72004
yielding image id 47821
yielding image id 72789
yielding image id 16773
yielding image id 49040
yielding image id 42453
yielding image id 23132
yielding image id 32060
yielding image id 29875
yielding image id 8060
yielding image id 82443
yielding image id 63832
yielding image id 81275
yielding image id 26483
yielding image id 40504
yielding image id 91192
yielding image id 23157
yielding image id 80312
yielding image id 93694
yielding image id 56612
yielding image id 44163
yielding image id 51174
yielding image id 25126
yielding image id 72496
yielding image id 91437
yielding image id 84178
yielding image id 10528
yielding image id 5651
yielding image id 33079
yielding image id 66552
yielding image id 20396
yielding image id 72853
yielding image id 80032
yielding image id 35085
yielding image id 22435
yielding image id 80387
yielding image id 16384
yielding image id 22735
yielding image id 21859
yielding image id 23110
yielding image id 17648
yieldin

yielding image id 31026
yielding image id 73275
yielding image id 73886
yielding image id 89032
yielding image id 31532
yielding image id 88185
yielding image id 81968
yielding image id 67440
yielding image id 61909
yielding image id 11144
yielding image id 61669
yielding image id 70749
yielding image id 61013
yielding image id 38264
yielding image id 29480
yielding image id 8197
yielding image id 22956
yielding image id 21170
yielding image id 83325
yielding image id 35219
yielding image id 16828
yielding image id 38338
yielding image id 83258
yielding image id 5565
yielding image id 27147
yielding image id 72463
yielding image id 30152
yielding image id 94021
yielding image id 24486
yielding image id 6366
yielding image id 39143
yielding image id 5298
yielding image id 86803
yielding image id 57201
yielding image id 40672
yielding image id 18877
yielding image id 7189
yielding image id 40907
yielding image id 92580
yielding image id 2756
yielding image id 55647
yielding image id 6093

yielding image id 32608
yielding image id 30617
yielding image id 91989
yielding image id 116
yielding image id 49588
yielding image id 41566
yielding image id 69311
yielding image id 41572
yielding image id 72144
yielding image id 44120
yielding image id 8774
yielding image id 34866
yielding image id 11243
yielding image id 1518
yielding image id 33417
yielding image id 5082
yielding image id 15640
yielding image id 38942
yielding image id 72409
yielding image id 38565
yielding image id 11030
yielding image id 49611
yielding image id 86184
yielding image id 10178
yielding image id 65860
yielding image id 32935
yielding image id 41844
yielding image id 335
yielding image id 71309
yielding image id 94272
yielding image id 43533
yielding image id 83563
yielding image id 63665
yielding image id 2767
yielding image id 70812
yielding image id 56904
yielding image id 31504
yielding image id 31378
yielding image id 57415
yielding image id 36897
yielding image id 31801
yielding image id 15292


yielding image id 38413
yielding image id 87222
yielding image id 54604
yielding image id 45364
yielding image id 6722
yielding image id 28849
yielding image id 64697
yielding image id 19191
yielding image id 5682
yielding image id 588
yielding image id 68604
yielding image id 55754
yielding image id 35843
yielding image id 84107
yielding image id 55596
yielding image id 12190
yielding image id 78115
yielding image id 445
yielding image id 71105
yielding image id 28184
yielding image id 19677
yielding image id 17997
yielding image id 30147
yielding image id 62660
yielding image id 25226
yielding image id 23626
yielding image id 89901
yielding image id 53389
yielding image id 14113
yielding image id 92816
yielding image id 2384
yielding image id 42419
yielding image id 74424
yielding image id 89316
yielding image id 81038
yielding image id 74910
yielding image id 41517
yielding image id 35337
yielding image id 72542
yielding image id 20614
yielding image id 36526
yielding image id 66423

yielding image id 82349
yielding image id 84251
yielding image id 74631
yielding image id 82040
yielding image id 22887
